In [2]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions, EntitiesOptions, CategoriesOptions, ConceptsOptions

In [3]:
from ibm_watson.natural_language_understanding_v1 import SyntaxOptions,SyntaxOptionsTokens

In [4]:
from ibm_watson import ApiException

In [100]:
from getpass import getpass
from scipy.stats import norm

In [6]:
from pandas_profiling import ProfileReport
import pickle

In /home/emmanuel/miniconda3/envs/toughenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emmanuel/miniconda3/envs/toughenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/emmanuel/miniconda3/envs/toughenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/emmanuel/miniconda3/envs/toughenv/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases

In [5]:
authenticator = IAMAuthenticator(getpass('Enter api token'))
natural_language_understanding = NaturalLanguageUnderstandingV1(version='2020-08-01',authenticator=authenticator)
natural_language_understanding.set_service_url(getpass('Enter url of service'))

Enter api token········
Enter url of service········


In [1]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('trabajos_datascience.csv')

In [8]:
df.head(5)

,titulo,empresa,descripcion
0,Desarrollador de Software sin Experiencia,Minsait,Somos una de las principales empresas globales...
1,REDACTOR DE PRODUCCIÓN EXP. EN TV,Grupo Milenio,ESTAMOS EN BUSCA DE TU TALENTO COMO REDACTOR (...
2,Gerente de inteligencia comercial,"Besins Healthcare México, S.A. de C.V.",Puesto: Gerente de inteligencia comercialOfrec...
3,Fraud Ops Analyst,Citibanamex,Publicada 13 de noviembre de 2020\n\nFraud Ops...
4,UNODC- Asistente de Proyecto,UNDP,Job Description\n\n\n\n\n\nAgency\n\n\n\nUN Of...


In [9]:
def syntax_analyze(text):
    response = natural_language_understanding.analyze(
        text=text,
        features=Features(syntax=SyntaxOptions(sentences=True,tokens=SyntaxOptionsTokens(part_of_speech=True))))
    return response.get_result()

In [10]:
def get_tokens(response,part_speech='PROPN'):
    
    for text in df['descripcion']:
        propn=[]
        for token in response['syntax']['tokens']:
            if token['part_of_speech']==part_speech:
                propn.append(token['text'])
    return propn

In [15]:
results=[]
for text in df['descripcion']:
    try:
        result=get_tokens(syntax_analyze(text))
    except (ApiException,ConnectionError) as e:
        print(e)
        result='NA'
    results.append(result)

ERROR:root:unsupported text language: gl
Traceback (most recent call last):
  File "/home/emmanuel/miniconda3/envs/toughenv/lib/python3.7/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: unsupported text language: gl, Code: 400 , X-global-transaction-id: d4a537f1-b8f7-4f92-a778-01bc8389a591


Error: unsupported text language: gl, Code: 400 , X-global-transaction-id: d4a537f1-b8f7-4f92-a778-01bc8389a591


In [7]:
#LOAD LISTS
with open('lista_tokens.dat','rb') as f:
    results=pickle.load(f)

In [16]:
df['tokens']=results

In [85]:
save_results=np.array(results)

/home/emmanuel/miniconda3/envs/toughenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [34]:
df

,titulo,empresa,descripcion
0,Desarrollador de Software sin Experiencia,Minsait,Somos una de las principales empresas globales...
1,REDACTOR DE PRODUCCIÓN EXP. EN TV,Grupo Milenio,ESTAMOS EN BUSCA DE TU TALENTO COMO REDACTOR (...
2,Gerente de inteligencia comercial,"Besins Healthcare México, S.A. de C.V.",Puesto: Gerente de inteligencia comercialOfrec...
3,Fraud Ops Analyst,Citibanamex,Publicada 13 de noviembre de 2020\n\nFraud Ops...
4,UNODC- Asistente de Proyecto,UNDP,Job Description\n\n\n\n\n\nAgency\n\n\n\nUN Of...
...,...,...,...
738,Junior NOC Analyst,Solera,The Role\nWe're on the hunt for a Junior NOC A...
739,Trade Compliance Sr. Analyst,Emerson,Trade Compliance Sr. Analyst\n\nRequisition ID...
740,Business Program Manager,Microsoft,The IC3 Calling team is on a mission to delive...
741,Geodis Data Analyst,Esprezza,Descripción\n.\nExperiencia y Conocimientos\nA...


In [10]:
artesanal='Python, R, Java, Scala, SQL, JavaScript, C++, Hadoop, Spark, PySpark, SparkR, PostgreSQL, MySQL, \
Transact SQL, MongoDB, Cassandra, t-sql, Machine learning, classification, clasificación, algoritmos predictivos,\
clustering, regresión, deep learning, Jupyter, RStudio'.lower().split(sep=', ')

In [63]:
nube='IBM Cloud data and AI, AWS, Azure, GCP, Oracle, Anaconda, Watson Studio, Watson Machine Learning, Amazon SageMaker, \
Oracle, Cloud Service, Anaconda Enterprise, Cloudant, CouchDB, Db2 Event Store, Data Warehousing, \
BigML, AlteryxInc, Mozenda, OnBase, BigQuery, Snowflake, kubernetes, bluemix, saas, paas, iaas, databricks, \
bigquery, lambda, alibaba, elastic, elasticsearch'.lower().split(sep=', ')

In [2]:
nube='IBM, AWS, Azure, GCP, Oracle, Anaconda, Watson, Amazon, SageMaker, \
Oracle, Cloud Service, Anaconda Enterprise, Cloudant, CouchDB, Db2 Event Store, Warehousing, \
BigML, AlteryxInc, Mozenda, OnBase, BigQuery, Snowflake, kubernetes, bluemix, saas, paas, iaas, databricks, \
bigquery, lambda, alibaba, elastic, elasticsearch, cloud, nube'.lower().split(sep=', ')

In [7]:
for i in nube:
    print(i,end= ', ')

ibm, aws, azure, gcp, oracle, anaconda, watson, amazon, sagemaker, oracle, cloud service, anaconda enterprise, cloudant, couchdb, db2 event store, warehousing, bigml, alteryxinc, mozenda, onbase, bigquery, snowflake, kubernetes, bluemix, saas, paas, iaas, databricks, bigquery, lambda, alibaba, elastic, elasticsearch, cloud, nube, 

In [24]:
def applicar(row,categoria):
    try:
        return set([token.lower() for token in row]).intersection(set(categoria))
    except TypeError as e:
        return {}

artesanales=[]
for result in results:
    artesanales.append(applicar(result,artesanal))
    
suma_art=0
for elem in artesanales:
    if elem:
        suma_art+=1

In [131]:
suma_art

351

In [80]:
nubes=[]
for result in results:
    nubes.append(applicar(result,nube))
    
suma_nube=0
for elem in nubes:
    if elem:
        suma_nube+=1

In [ ]:
167

In [108]:
suma_nube

216

In [92]:
p=suma_nube/df.shape[0]

In [95]:
p*100

29.071332436069987

In [101]:
n=df.shape[0] #tenemos muchas observaciones, entonces es adecuado usar aprox. de limites distribucionales.
p_estimate=suma_nube/n
alpha=0.05 # nivel de confianza del 95%
def conf_interval(alpha,p_estim):
    M=norm.ppf(1-alpha/2)
    SE=np.sqrt(p_estimate*(1-p_estimate)/n)
    return np.array([p_estimate-M*SE,p_estimate+M*SE])*100

In [111]:
15/60

0.25

In [102]:
conf_interval(alpha,p_estimate)

array([25.80622818, 32.3364367 ])

In [81]:
suma_nube

216

In [21]:
results[569]

['DESARROLLADOR',
 'FULLSTACK',
 'Home',
 'officeProgramación',
 'Python',
 'CLOUD',
 'GCP',
 'App',
 'Engine',
 'Kubernetes',
 'big',
 'query',
 'APIS',
 'Google',
 'Cloud',
 'Platform',
 'Java',
 'libresPrestaciones',
 'IMSSInfonavit',
 'mayoresSeguro',
 'vidaTarjeta',
 'descuentosGastos',
 'funerariosSeguro',
 'Odontológico',
 'oftalmológicoTipo',
 'IndefinidoSalario',
 'mesExperiencia',
 'GOOGLE',
 'CLOUD',
 'PLATFORM',
 'Python']

In [55]:
n=70
np.random.seed(10)
index_sample=np.random.choice(range(0,len(results)),size=n,replace=False)

In [56]:
index_sample

array([476, 358, 417, 194, 375, 569, 567, 294, 196, 179, 242, 354, 322,
       620, 600, 494, 152, 230, 678, 627, 437,  27, 696, 195, 555,  45,
       163, 408, 302,  94, 477, 292, 157, 217,  43, 374, 667, 220, 637,
       617, 508, 329, 448, 559, 389, 495, 309, 202, 367, 580, 575, 345,
        64, 299, 483, 599, 452, 584, 131, 726, 116, 143, 396, 285, 522,
       254, 278, 222, 680, 475])

In [57]:
array_results=np.array(results)

/home/emmanuel/miniconda3/envs/toughenv/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [134]:
suma_nube

167

In [38]:
from nltk.corpus import stopwords

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

documents = ["This little kitty came to play when I was eating at a restaurant.",
             "Merley has the best squooshy kitten belly.",
             "Google Translate app is incredible.",
             "If you open 100 tab in google you get a smiley face.",
             "Best cat photo I've ever taken.",
             "Climbing ninja cat.",
             "Impressed with google map feedback.",
             "Key promoter extension for Google Chrome."]

# documents=list(df.descripcion)

# vectorizer = TfidfVectorizer(stop_words=stopwords.words('spanish'))
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

In [107]:
model.cluster_centers_.argsort()

array([[ 0, 29, 28, 27, 25, 22, 20, 19, 31, 15, 14, 13, 16, 11, 10,  9,
         6,  1, 12,  2,  3,  8, 17, 18, 24,  4, 26, 30, 32, 23, 21,  7,
         5],
       [32, 30,  4,  5,  7,  8, 26, 24, 23, 21, 18, 17,  3, 29, 27, 22,
         0, 10, 20, 28,  2, 16, 15, 25,  9,  6, 14, 19, 13, 11,  1, 31,
        12]])

In [106]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["chrome browser to open."])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["My cat is hungry."])
prediction = model.predict(Y)
print(prediction)
 

Top terms per cluster:
Cluster 0:
 cat
 climbing
 ninja
 photo
 ve
 taken
 restaurant
 came
 play
 little
Cluster 1:
 google
 translate
 app
 feedback
 impressed
 map
 incredible
 chrome
 extension
 promoter


Prediction
[1]
[0]
